In [7]:
import os
import numpy as np
from towbintools.foundation.image_handling import read_tiff_file
from towbintools.foundation.file_handling import get_dir_filemap, add_dir_to_experiment_filemap
from tifffile import imwrite
from skimage.measure import label

In [8]:
img_stack_dir = "/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/stardist/database/raw/"
mask_stack_dir = "/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/stardist/database/corrected_mask/"

img_stack_files = sorted([f for f in os.listdir(img_stack_dir) if f.endswith(".tiff")])
mask_stack_files = sorted([f for f in os.listdir(mask_stack_dir) if f.endswith(".tiff")])

filemap = get_dir_filemap(img_stack_dir)
filemap = add_dir_to_experiment_filemap(filemap, mask_stack_dir, "MaskPath")

# keep only the rows where the mask file exists
filemap = filemap[filemap["MaskPath"] != ""]

print(f'Found {len(filemap)} image-mask pairs')

Found 6 image-mask pairs


In [9]:
training_img_dir = "/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/stardist/images/"
training_mask_dir = "/mnt/towbin.data/shared/spsalmon/20240524_161257_273_LIPSI_40x_397_405_no_crash/stardist/masks/"

os.makedirs(training_img_dir, exist_ok=True)
os.makedirs(training_mask_dir, exist_ok=True)

# for each mask, go through its plane, and save the corresponding image plane if the mask has more than 10 objects
for idx, row in filemap.iterrows():
    img_file = row["ImagePath"]
    mask_file = row["MaskPath"]

    mask = read_tiff_file(mask_file)

    for i, mask_plane in enumerate(mask):
        if len(np.unique(mask_plane)) > 10:
            img_plane = read_tiff_file(img_file)[i]
            img_basename = os.path.basename(img_file).replace(".ome.tiff", f"_plane{i}.tiff")
            mask_basename = os.path.basename(mask_file).replace(".ome.tiff.tiff", f"_plane{i}.tiff")

            mask_plane = mask_plane.astype(np.uint16)
            mask_plane = label(mask_plane).astype(np.uint16)

            img_path = os.path.join(training_img_dir, img_basename)
            mask_path = os.path.join(training_mask_dir, mask_basename)

            imwrite(img_path, img_plane, compression="zlib")
            imwrite(mask_path, mask_plane, compression="zlib")